In [2]:
import torch
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from safetensors.torch import load_file

# 基础 SD1.5 模型，可以换成你需要的
# base_model = "runwayml/stable-diffusion-v1-5"
base_model = "/home/mlsnrs/data/common_model/models--runwayml--stable-diffusion-v1-5/snapshots/451f4fe16113bff5a5d2269ed5ad43b0592e9a14"

# LoRA 权重路径 (你保存的 .safetensors 或者 .bin 文件)
lora_path = "./output/pokemon_small/pokemon_small.safetensors"

# 加载 pipeline
device = "cuda:7"
pipe = StableDiffusionPipeline.from_pretrained(
    base_model,
    torch_dtype=torch.float16
).to(device)

# ===== 加载 LoRA 权重 =====
# 如果你是 safetensors 格式
state_dict = load_file(lora_path)

# 遍历 LoRA 权重，注入到 UNet / TextEncoder
def load_lora_weights(pipe, state_dict, alpha=1.0):
    visited = []
    for key in state_dict:
        # 例子: lora_unet_down_blocks_0_attentions_0_query.lora_down.weight
        if "lora_down" in key or "lora_up" in key:
            layer_infos = key.split(".")[0:-2]
            curr_layer = pipe.unet
            for info in layer_infos:
                if info.isdigit():
                    curr_layer = curr_layer[int(info)]
                else:
                    curr_layer = getattr(curr_layer, info)
            if "lora_down" in key:
                down_weight = state_dict[key]
                up_weight = state_dict[key.replace("lora_down", "lora_up")]
                # LoRA 权重注入
                weight = (up_weight @ down_weight) * alpha
                curr_layer.weight.data += weight.to(curr_layer.weight.data.device, dtype=curr_layer.weight.data.dtype)
                visited.append(key)
    print(f"Loaded LoRA weights: {len(visited)//2} layers injected")

load_lora_weights(pipe, state_dict, alpha=0.8)

# ===== 推理 =====
prompt = "a cute pokemon, digital art, highly detailed"
image = pipe(prompt, num_inference_steps=25, guidance_scale=7.5).images[0]

# 保存图片
# image.save("output.png")

plt.imshow(image)
plt.axis("off")
plt.show()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:  86%|████████▌ | 6/7 [00:22<00:04,  4.05s/it]/home/mlsnrs/data/miniconda3/envs/cpy_sd/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Loading pipeline components...: 100%|██████████| 7/7 [00:22<00:00,  3.23s/it]


AttributeError: 'UNet2DConditionModel' object has no attribute 'lora_te_text_model_encoder_layers_0_mlp_fc1'